In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import brown, stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.util import ngrams
import itertools
import networkx as nx
from collections import Counter
import json

nltk.download('punkt')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = stopwords.words('english')

np.set_printoptions(suppress=True)

[nltk_data] Downloading package punkt to /Users/r0g06z5/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /Users/r0g06z5/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/r0g06z5/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/r0g06z5/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Word Sense Disambiguation

The problem of Word Sense Disambiguation (WSD) is identifying the correct sense of word being used in a sentence. In English Language, there can be multiple senses of the same word. 

For example - Apple in a sentence can be used as a fruit or as the name of company.

In [2]:
ambiguous_word = 'ash'
word_senses = wn.synsets(ambiguous_word)
print('Word:', ambiguous_word)
print('Number of senses:', len(word_senses))
print()
for i, word_sense in enumerate(word_senses):
    print(f'Sense {i+1}:', word_sense.definition())

Word: ash
Number of senses: 4

Sense 1: the residue that remains when something is burned
Sense 2: any of various deciduous pinnate-leaved ornamental or timber trees of the genus Fraxinus
Sense 3: strong elastic wood of any of various ash trees; used for furniture and tool handles and sporting goods such as baseball bats
Sense 4: convert into ashes


### Basic Algorithm

Given a sentence and an ambiguous word in it, we are interested in finding the sense of that word. So, we create **sense bags** which is collection of features (hyponyms/hypernyms/definitions or others) for each sense of ambiguous word and a **context bag** which is collection of features for all the words in sentence excluding ambiguous word.

We calculate overlap between context bag and various sense bags (one for each sense). We consider that sense which has maximum overlap.

1. Hyponyms are words of more specific meaning than given word. Example - Apple is hyponym of fruit.
2. Hypernyms are words of broader meaning than given word. Example - Fruit is hypernym of Apple.

In [3]:
def get_sense_context_bag(sample_sentence, ambiguous_word, feature):
    words = sample_sentence.split(' ')
    words = [w.lower() for w in words]
    words.remove(ambiguous_word)

    sense_bag = []
    context_bag = []
    for word in words:
        word_senses = wn.synsets(word)

        if feature == 'hyponym':
            word_hyponyms = list(itertools.chain(*map(lambda x: x.hyponyms(), word_senses)))
            context_bag += word_hyponyms
        if feature == 'hypernym':
            word_hypernyms = list(itertools.chain(*map(lambda x: x.hypernyms(), word_senses)))
            context_bag += word_hypernyms

    ambiguous_word_senses = wn.synsets(ambiguous_word)
    if feature=='hyponym':
        sense_bag = list(map(lambda x: x.hyponyms(), ambiguous_word_senses))
    if feature=='hypernym':
        sense_bag = list(map(lambda x: x.hypernyms(), ambiguous_word_senses))
    return sense_bag, set(context_bag)  

In [4]:
def get_sense(sample_sentence, ambiguous_word, feature):
    sense_bag, context_bag = get_sense_context_bag(sample_sentence, ambiguous_word, feature)

    # count overlap between context bag and various sense bags, one for each sense
    count_overlap = list(map(lambda x: len(set(x) and context_bag), sense_bag))
    sense_idx = count_overlap.index(max(count_overlap))
    sense_dfn = wn.synsets(ambiguous_word)[sense_idx].definition()
    return sense_dfn

In [5]:
sample_sentence = 'I am eating apple'
ambiguous_word = 'apple'
feature = 'hyponym'
sense_dfn = get_sense(sample_sentence, ambiguous_word, feature) 
print('Ambiguous word:', ambiguous_word)   
print('Definition:', sense_dfn)

Ambiguous word: apple
Definition: fruit with red or yellow or green skin and sweet to tart crisp whitish flesh


### Lesk's Algorithm

Creating sense bags and context bag based on definition of words.

In [6]:
def preprocess(sense):
    words = sense.definition().split()
    words = [word.lower() for word in words if word.isalnum() and word not in stopwords]
    return words

def Lesk_algorithm(ambiguous_word, context):
    sense_bag = list(map(lambda x: preprocess(x), wn.synsets(ambiguous_word)))

    context_words = context.strip().split(' ')
    context_bag = []
    for context_word in context_words:
        context_bag += list(itertools.chain(*map(lambda x: preprocess(x), wn.synsets(context_word))))
    context_bag = set(context_bag)
    
    count_overlap = list(map(lambda x: len(set(x) and context_bag), sense_bag))
    sense_idx = count_overlap.index(max(count_overlap))
    sense_dfn = wn.synsets(ambiguous_word)[sense_idx].definition()
    return sense_dfn

In [7]:
ambiguous_word = 'ash'
context = 'coal'
sense_dfn = Lesk_algorithm(ambiguous_word, context)
print('Ambiguous word:', ambiguous_word)   
print('Definition:', sense_dfn)

Ambiguous word: ash
Definition: the residue that remains when something is burned


### Walker's Algorithm

Creating sense bags and context bag based on thesaurus.

In [8]:
def get_thesaurus(sense):
    return list(set(itertools.chain(*map(lambda x: x.lemma_names(), sense.hypernyms() + sense.hyponyms()))))

def Walker_algorithm(ambiguous_word, context):
    thesaurus_senses = list(map(lambda x: get_thesaurus(x), wn.synsets(ambiguous_word)))
    assert len(thesaurus_senses) == len(wn.synsets(ambiguous_word))

    context_words = context.strip().split(' ')
    thesaurus_contexts = []
    for context_word in context_words:
        thesaurus_contexts += [list(set(itertools.chain(*map(lambda x: get_thesaurus(x), wn.synsets(context_word)))))]
    assert len(thesaurus_contexts) == len(context_words)

    count_overlaps = []
    for thesaurus_sense in thesaurus_senses:
        count_overlap = 0
        for thesaurus_context in thesaurus_contexts:
            if len(set(thesaurus_sense).intersection(set(thesaurus_context))) > 0:
                count_overlap +=1
        count_overlaps.append(count_overlap)
        
    print('Number of overlaps:', count_overlaps)
    sense_idx = count_overlaps.index(max(count_overlaps))
    sense_dfn = wn.synsets(ambiguous_word)[sense_idx].definition()
    return sense_dfn

ambiguous_word = 'burn'
context = 'coal fire flame residue wood combust'

print('Ambiguous word:', ambiguous_word)  
sense_dfn = Walker_algorithm(ambiguous_word, context)
print('Definition:', sense_dfn)

Ambiguous word: burn
Number of overlaps: [0, 0, 2, 0, 1, 2, 1, 3, 1, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 2]
Definition: undergo combustion


### WSD using Random Walk algorithm

The problem with the above approaches is that we are assuming that the words in context are not ambiguous i.e. context words are capturing the right sense. However, it's possible that there are multiple ambiguous words in a sentence which we need to disambiguate all together.

For this, we use Random Walk approach to calculate **PageRank importances**. The idea is that in a sentence some senses of ambiguous words correlate together and they all will contribute to each other in getting higher PageRank importance. Once we reach convergence according to PageRank criteria (limiting distribution of Markov Chain), for each ambiguous word, we consider sense with highest importance and assume it to be the right sense given sentence.

In [9]:
# get senses for words in sentence
def get_senses(sentence):
    words = sentence.split(' ')
    senses = []
    word_sense_labels = []
    for word in words:
        senses.append(wn.synsets(word))
        word_sense_labels += list(map(lambda x: word + '-' + x.name(), wn.synsets(word)))
    senses = list(set(itertools.chain(*senses)))

    assert len(word_sense_labels) == len(senses)
    sense2node = dict(zip(senses, range(len(senses))))
    node2sense = {v:k for k,v in sense2node.items()}
    return senses, word_sense_labels, sense2node, node2sense

# adding nodes
def add_nodes(G, senses):
    nodes = range(len(senses))
    G.add_nodes_from(nodes)
    return G

# adding edges
def add_edges(G, senses, sense2node):
    for sense1 in senses:
        for sense2 in senses:
            if sense1!=sense2:
                def1 = set(preprocess(sense1))
                def2 = set(preprocess(sense2))
                similarity = len(def1.intersection(def2))
                
                node1 = sense2node[sense1]
                node2 = sense2node[sense2]
                G.add_edge(node1, node2, weight=similarity)
    return G

# normalizing edge weights
def normalize_edge_weights(G):
    for node in G.nodes():
        if len(G.successors(node)) > 0:
            total_weight = 0
            for neighbor in G.successors(node):
                total_weight += G[node][neighbor]['weight']
            if total_weight!=0:
                for neighbor in G.successors(node):
                    G[node][neighbor]['weight']/= total_weight
    return G

# removing isolated nodes
# edges to/from removed nodes are removed as well
def remove_isolated_nodes(G):
    nodes = G.nodes()
    for node in nodes:
        if len(G.successors(node))==0 and len(G.predecessors(node))==0:
            G.remove_node(node)   
    return G                 

In [10]:
def make_pagerank_matrix(G, alpha):
    n_nodes = len(G.nodes())

    # building adjacent matrix
    adj_matrix = np.zeros(shape=(n_nodes, n_nodes))
    for edge in G.edges():
        adj_matrix[edge[0], edge[1]] = 1

    # building transition probability matrix
    tran_matrix = adj_matrix / np.sum(adj_matrix, axis=1).reshape(-1,1)

    # building random surfer matrix
    random_surf = np.ones(shape = (n_nodes, n_nodes)) / n_nodes    

    # building transition matrix for absorbing nodes
    absorbing_nodes = np.zeros(shape = (n_nodes,))
    for node in G.nodes():
        if len(G.successors(node))==0:
            absorbing_nodes[node] = 1
    absorbing_node_matrix = np.outer(absorbing_nodes, np.ones(shape = (n_nodes,))) / n_nodes

    # stochastic matrix
    stochastic_matrix = tran_matrix + absorbing_node_matrix

    # pagerank matrix
    pagerank_matrix = alpha * stochastic_matrix + (1-alpha) * random_surf
    return pagerank_matrix

In [11]:
def random_walk(G, alpha, n_iter):
    n_nodes = len(G.nodes())
    initial_state = np.ones(shape=(n_nodes,)) / n_nodes
    pagerank_matrix = make_pagerank_matrix(G, alpha)

    new_initial_state = initial_state
    print('Running random walk..')
    NORM = []
    for i in range(n_iter):
        final_state = np.dot(np.transpose(pagerank_matrix), new_initial_state)
        
        prev_initial_state = new_initial_state
        new_initial_state = final_state
        L2 = np.linalg.norm(new_initial_state-prev_initial_state)
        NORM.append(L2)
        if np.allclose(new_initial_state, prev_initial_state):
            break
    print('Complete','\n')
    return final_state

In [12]:
def get_final_senses(sentence, pagerank_importances, word_sense_labels):
    word_senses = np.array(list(map(lambda x: x.split('-'), word_sense_labels)))
    words = word_senses[:, 0]
    senses = word_senses[:, 1]

    for word in sentence.split(' '):
        idxs = np.where(words==word)[0]
        word_pr = pagerank_importances[idxs]
        sense_idx = np.where(word_pr == max(word_pr))[0][0]
        sense_names = senses[idxs][sense_idx]
        print(f'Senses for word - {word}:', sense_names)

In [13]:
def run(sentence, alpha, n_iter):
    senses, word_sense_labels, sense2node, node2sense = get_senses(sentence)
    G = nx.DiGraph()

    # adding nodes
    G = add_nodes(G, senses)

    # adding edges
    G = add_edges(G, senses, sense2node)

    print('Number of nodes:', len(G.nodes()))
    print('Number of edges:', len(G.edges()))
    print() 
    
    # remove nodes with no edges
    G = remove_isolated_nodes(G)

    # normalizing edge weights
    G = normalize_edge_weights(G)

    # running Random Walk
    pagerank_importances = random_walk(G, alpha, n_iter)

    # get final senses
    get_final_senses(sentence, pagerank_importances, word_sense_labels)

In [14]:
alpha = 0.1
n_iter = 1000

sentence = 'run walk fast speed'
run(sentence, alpha, n_iter)

Number of nodes: 99
Number of edges: 9702

Running random walk..
Complete 

Senses for word - run: run.n.13
Senses for word - walk: walk.n.01
Senses for word - fast: fast.n.01
Senses for word - speed: travel_rapidly.v.01
